In [107]:
%matplotlib inline

from compare_bmp import compare_images
from PIL import Image, ImageFilter
from matplotlib.pyplot import imshow
import numpy as np

We'll start with this image:
![img1](images/original/image.bmp)

In [108]:
# let's get our message set up
message = list('this is a message')
# convert to binary representation
message = ['{:07b}'.format(ord(x)) for x in message]
print(message)
# split the binary into 
message = [[bit for bit in x] for x in message]
print(message)
# flatten it and convert to integers
message = [int(bit) for sublist in message for bit in sublist]
print(message)
originalMessage = message

['1110100', '1101000', '1101001', '1110011', '0100000', '1101001', '1110011', '0100000', '1100001', '0100000', '1101101', '1100101', '1110011', '1110011', '1100001', '1100111', '1100101']
[['1', '1', '1', '0', '1', '0', '0'], ['1', '1', '0', '1', '0', '0', '0'], ['1', '1', '0', '1', '0', '0', '1'], ['1', '1', '1', '0', '0', '1', '1'], ['0', '1', '0', '0', '0', '0', '0'], ['1', '1', '0', '1', '0', '0', '1'], ['1', '1', '1', '0', '0', '1', '1'], ['0', '1', '0', '0', '0', '0', '0'], ['1', '1', '0', '0', '0', '0', '1'], ['0', '1', '0', '0', '0', '0', '0'], ['1', '1', '0', '1', '1', '0', '1'], ['1', '1', '0', '0', '1', '0', '1'], ['1', '1', '1', '0', '0', '1', '1'], ['1', '1', '1', '0', '0', '1', '1'], ['1', '1', '0', '0', '0', '0', '1'], ['1', '1', '0', '0', '1', '1', '1'], ['1', '1', '0', '0', '1', '0', '1']]
[1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0

In [109]:
# first, open the original image and hide a message in it
imgpath = 'images/original/image.bmp'

img = Image.open(imgpath)

# we'll use simple repetition as a very rudimentary ECC to try to maintain integrity
# each bit of the message will be repeated 6 times - the two LSBs of the R,G, and B values of one pixel
#imgArray = np.copy(np.asarray(img))[:1]
imgArray = list(np.asarray(img))

def set_bit(val, bitNo, bit):
    """ given a value, which bit in the value to set, and the actual bit (0 or 1) to set, return the new value """
    mask = 1 << bitNo
    val &= ~mask
    if bit:
        val |= mask
    return val

msgIndex = 0
newImg = []

for row in imgArray:
    newRow = []
    for pixel in row:
        newPixel = []
        for val in pixel:
            # iterate through RGB values, one at a time
            #print(bin(r))
            if msgIndex >= len(message):
                setTo = 0
            else:
                setTo = message[msgIndex]
            val = set_bit(val, 0, setTo)
            val = set_bit(val, 1, setTo)
            val = set_bit(val, 2, setTo)
            
            # check to make sure that the last 3 bits are the same thing - should not be mixed
            """if not '{:08b}'.format(val)[5:] == '000' and not '{:08b}'.format(val)[5:] == '111':
                print('{:08b}'.format(val)[5:])
                raise ValueError('stuff broke')"""
                
            newPixel.append(val)
        msgIndex += 1
        newRow.append(newPixel)
    newImg.append(newRow)

arr = np.array(newImg, np.uint8)
im = Image.fromarray(arr)
im.save("image_steg.bmp")

How does the image look now?

![image with hidden data](image_steg.bmp)

Awesome! Doesn't look like anything's wrong.

In [110]:
# now let's blur the image to destroy some data
blurredpath = 'image_blurred.bmp'

img = Image.open("image_steg.bmp")
blurred = img.copy().filter(ImageFilter.BLUR)

blurred.save(blurredpath)

And here it is now that we've blurred it:
![img_blurred](images/image_blurred.bmp)

In [111]:
# last, open the image and apply our ECC strategy to see if the message made it through

blurredImg = Image.open(blurredpath)

imgArray = list(np.asarray(blurredImg))
            
origMessage = message[:20]
print(origMessage)

message = []

for val in imgArray[0][:20]:
    print([bin(x) for x in val])

for row in imgArray:
    for pixel in row:
        count = {"0": 0, "1": 0}
        for val in pixel:
            # iterate through RGB values of the pixel, one at a time
            # convert the R, G, or B value to a byte string
            byte = '{:08b}'.format(val)
            # then, for each of the last n bits...
            for i in [-1, -2, -3]:
                # try to get an actual 1 or 0 integer from it
                try:
                    bit = int(byte[i])
                except:
                    print(bin(val))
                    raise

                # sum up the bits we've seen
                if bit == 0:
                    count["0"] += 1
                elif bit == 1:
                    count["1"] += 1
                else:
                    print("WAT")
                    
        # and once we've seen them all, decide which we should go with
        if count["1"] > count["0"]:
            message.append(1)
        else:
            message.append(0)

print(message[:20])

[1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0]
['0b11101111', '0b11101111', '0b11101111']
['0b11101111', '0b11101111', '0b11101111']
['0b11101111', '0b11110111', '0b11101111']
['0b11101000', '0b11110000', '0b11101000']
['0b11101111', '0b11110111', '0b11101111']
['0b11101000', '0b11101000', '0b11101000']
['0b11101000', '0b11101000', '0b11101000']
['0b11101111', '0b11110111', '0b11100111']
['0b11101111', '0b11110111', '0b11101111']
['0b11110000', '0b11110000', '0b11101000']
['0b11110111', '0b11110111', '0b11101111']
['0b11110000', '0b11110000', '0b11101000']
['0b11110000', '0b11110000', '0b11101000']
['0b11110000', '0b11110000', '0b11101000']
['0b11110111', '0b11110111', '0b11101111']
['0b11110111', '0b11110111', '0b11101111']
['0b11110000', '0b11110000', '0b11110000']
['0b11110111', '0b11110111', '0b11110111']
['0b11110000', '0b11110000', '0b11110000']
['0b11110000', '0b11110000', '0b11110000']
[1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0]


In [112]:
from math import floor
# let's see what the message says!

outputMessage = []

originalMessage = list('this is a message')
originalMessage = [ord(x) for x in originalMessage]

for i in range(floor(len(message)/7)):
    start = i * 7
    # this is gross
    char = message[start:start+7]
    char = '0b' + ''.join([str(c) for c in char])
    #print(char)
    outputMessage.append(int(char, 2))

print(originalMessage)
print(outputMessage)

print("ORIGINAL VS. OUTPUT")
print("===================")
print(''.join([chr(c) for c in originalMessage]))
print(''.join([chr(c) for c in outputMessage]))

[116, 104, 105, 115, 32, 105, 115, 32, 97, 32, 109, 101, 115, 115, 97, 103, 101]
[116, 104, 105, 115, 32, 105, 115, 32, 97, 32, 109, 101, 115, 115, 97, 103, 101, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 49, 25, 15, 127, 64, 127, 99, 9, 127, 122, 0, 7, 126, 30, 60, 65, 127, 127, 127, 107, 115, 99, 32, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 64, 62, 63, 12, 0, 6, 4, 15, 1, 124, 0, 96, 31, 127, 0, 60, 0, 48, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 55, 60, 16, 0, 0, 126, 63, 127, 96, 0, 29, 109, 11, 71, 79, 0, 15, 16, 27, 78, 0, 63, 123, 127, 124, 15, 127, 84, 0, 64, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 3, 79, 53, 120, 31, 112, 65, 112, 0, 63, 115, 126, 3, 127, 9,